In [1]:
# from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
# from category_encoders import *
# from sklearn.preprocessing import OrdinalEncoder
# from sklearn.metrics import accuracy_score, f1_score
# import lightgbm as lgbm
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [2]:
def update_col(df_col):
    if df_col.dtype=='int64': d['n_numerical']+=1
    if df_col.dtype=='float64': d['n_numerical']+=1
    if df_col.dtype=='object': 
        d['n_categorical']+=1                                                        # increase n_categorical count
        d['highest_cardinality']=max(df_col.nunique(),d['highest_cardinality'])    # get highest cardinality
    
def update_dict(df,y):
    
    for col in df.columns:
        update_col(df[col])
    
    d['n_samples'], d['n_features']=df.shape[0],df.shape[1]    
    d['nulls']=round(df.isna().sum().sum()/(d['n_samples']*d['n_features']),3)
    
    d['n_classes']=y.nunique()
    freq=y.value_counts()
    d['classes']=list(freq.index)
    d['class_ratio']=list(round(freq/freq.sum(),4))
    
    for ratio in y.value_counts()/y.value_counts().sum():
        equality_of_classes=100/d['n_classes']
        limit=2*equality_of_classes/10
        if ((ratio> (equality_of_classes+limit) ) | (ratio< (equality_of_classes-limit) )): d['imbalanced']=1

    
    return d

def initialize_dict():
    d={'n_samples':0,'n_features':0,'n_categorical':0,'n_numerical':0,'highest_cardinality':0,'nulls':0,'n_classes':0,'classes':0,'class_ratio':0,'imbalanced':0,'domain':None,'xgboost_acc':0,'catboost_acc':0,'lgbm_acc':0,'RF_acc':0}
    return d
 

In [105]:
df_meta=pd.DataFrame(columns=['dataset','n_samples','n_features','n_categorical','n_numerical','highest_cardinality','nulls','n_classes','classes','class_ratio','imbalanced','domain','xgboost_acc','catboost_acc','lgbm_acc','RF_acc'])

### adult 

In [ ]:
df=pd.read_csv('adult.data',header=None)
df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


In [ ]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]
d=initialize_dict()
df_meta=df_meta.append(update_dict(X,y),ignore_index=True)
df_meta.iloc[-1,0]='adult'

### beer reviews 

In [ ]:
df=pd.read_csv('F:/data/beer/beer_reviews.csv')
df.head(3)

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215


In [ ]:
X=df.loc[:,['beer_name','review_aroma','review_appearance','review_palate','review_taste']]
y=df.loc[:,'beer_style']
d=initialize_dict()
df_meta=df_meta.append(update_dict(X,y),ignore_index=True)
df_meta.iloc[-1,0]='beer_review'

### traffic violations 

In [ ]:
df=pd.read_csv('F:/data/traffic_violations/Traffic_Violations.csv')
df.head(3)

C:\Users\nishant\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (19,20,21,22,23,24,25,34) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SeqID,Date Of Stop,Time Of Stop,Agency,SubAgency,Description,Location,Latitude,Longitude,Accident,...,Charge,Article,Contributed To Accident,Race,Gender,Driver City,Driver State,DL State,Arrest Type,Geolocation
0,02ccadf1-7ebd-48d8-a793-913e3198f52a,07/30/2019,22:15:00,MCP,"2nd District, Bethesda",EXCEEDING THE POSTED SPEED LIMIT OF 35 MPH,RIVER RD/ROYAL DOMINION DR,38.990102,-77.151645,No,...,21-801.1,Transportation Article,False,ASIAN,F,FAIRFAX,VA,VA,Q - Marked Laser,"(38.9901016666667, -77.151645)"
1,3b060228-d664-4480-9314-6188ee8ac891,07/07/2019,22:54:00,MCP,"4th District, Wheaton",FAILURE TO STOP AT STOP SIGN,ENNALLS AVE @ GEORGIA AVE,39.040360,-77.051137,No,...,21-707(a),Transportation Article,False,HISPANIC,M,SILVER SPRING,MD,MD,A - Marked Patrol,"(39.04036, -77.0511366666667)"
2,d4dd55c0-adb0-434e-b5d9-d8346b0a4080,07/10/2019,16:00:00,MCP,"1st District, Rockville",HEADLIGHT INOPERATIVE,SHADY GROVE @ TUPELO,0.000000,0.000000,No,...,11-393.9,Maryland Rules,False,HISPANIC,M,ALEXANDRIA,VA,VA,A - Marked Patrol,"(0.0, 0.0)"


In [ ]:
X=df.loc[:,[ 'Description','Belts','Property Damage','Fatal','Commercial License','HAZMAT','Commercial Vehicle','Alcohol','Work Zone','Year','Race','Gender','Arrest Type' ]]
y=df.loc[:,'Violation Type']
d=initialize_dict()
df_meta=df_meta.append(update_dict(X,y),ignore_index=True)
df_meta.iloc[-1,0]='traffic_violations'
df_meta

,dataset,n_samples,n_features,n_categorical,n_numerical,highest_cardinality,nulls,n_classes,classes,class_ratio,imbalanced,domain,xgboost_acc,catboost_acc,lgbm_acc,RF_acc
0,adult,32561,14,8,6,42,0.000000,2,"[ <=50K, >50K]","[0.7592, 0.2408]",1,None,0,0,0,0
1,beer_review,1586614,5,1,4,56857,0.000000,104,"[American IPA, American Double / Imperial IPA,...","[0.0741, 0.0542, 0.04, 0.0341, 0.032, 0.0318, ...",1,None,0,0,0,0
2,traffic_violations,1655626,13,12,1,14707,0.078489,4,"[Warning, Citation, ESERO, SERO]","[0.5043, 0.4475, 0.0476, 0.0005]",1,None,0,0,0,0


### road accidents- vehicles

In [ ]:
df=pd.read_csv('F:/data/road_accident/dftRoadSafetyData_Vehicles_2018.csv')
df.head(3)

C:\Users\nishant\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Accident_Index,Vehicle_Reference,Vehicle_Type,Towing_and_Articulation,Vehicle_Manoeuvre,Vehicle_Location-Restricted_Lane,Junction_Location,Skidding_and_Overturning,Hit_Object_in_Carriageway,Vehicle_Leaving_Carriageway,...,Journey_Purpose_of_Driver,Sex_of_Driver,Age_of_Driver,Age_Band_of_Driver,Engine_Capacity_(CC),Propulsion_Code,Age_of_Vehicle,Driver_IMD_Decile,Driver_Home_Area_Type,Vehicle_IMD_Decile
0,2018010080971,1,9,0,18,0,0,0,0,0,...,6,1,32,6,1995,2,5,8,1,8
1,2018010080971,2,8,0,18,0,0,0,0,0,...,6,1,48,8,1798,8,6,1,1,1
2,2018010080973,1,9,0,18,0,2,0,0,0,...,6,3,-1,-1,-1,-1,-1,-1,-1,-1


In [ ]:
X=df.loc[:,[ 'Was_Vehicle_Left_Hand_Drive?','Engine_Capacity_(CC)','Age_of_Vehicle','Propulsion_Code' ]]
y=df.loc[:,'Age_Band_of_Driver']
d=initialize_dict()
df_meta=df_meta.append(update_dict(X,y),ignore_index=True)
df_meta.iloc[-1,0]='road_accidents_vehicles'

### bank marketing 

In [ ]:
df=pd.read_csv('F:/data/bank marketing/bank-additional-full.csv',delimiter=';')
df.head(3)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [ ]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]
d=initialize_dict()
df_meta=df_meta.append(update_dict(X,y),ignore_index=True)
df_meta.iloc[-1,0]='bank marketing'

### kick

In [ ]:
df=pd.read_csv('F:/data/kick/training.csv')
df.head(3)

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
0,1,0,12/7/2009,ADESA,2006,3,MAZDA,MAZDA3,i,4D SEDAN I,...,11597.0,12409.0,NaN,NaN,21973,33619,FL,7100.0,0,1113
1,2,0,12/7/2009,ADESA,2004,5,DODGE,1500 RAM PICKUP 2WD,ST,QUAD CAB 4.7L SLT,...,11374.0,12791.0,NaN,NaN,19638,33619,FL,7600.0,0,1053
2,3,0,12/7/2009,ADESA,2005,4,DODGE,STRATUS V6,SXT,4D SEDAN SXT FFV,...,7146.0,8702.0,NaN,NaN,19638,33619,FL,4900.0,0,1389


In [ ]:
X=df.iloc[:,2:]
y=df.iloc[:,1]
d=initialize_dict()
df_meta=df_meta.append(update_dict(X,y),ignore_index=True)
df_meta.iloc[-1,0]='kick'

### poker hand 

In [ ]:
df=pd.read_csv('F:/data/poker/poker-hand-training-true.data',header=None)
df.head(3)

,0,1,2,3,4,5,6,7,8,9,10
0,1,10,1,11,1,13,1,12,1,1,9
1,2,11,2,13,2,10,2,12,2,1,9
2,3,12,3,11,3,13,3,10,3,1,9


In [ ]:
df[0]=df[0].astype(str)
df[2]=df[2].astype(str)
df[4]=df[4].astype(str)
df[6]=df[6].astype(str)
df[8]=df[8].astype(str)
df[10]=df[10].astype(str)

X=df.iloc[:,:-1]
y=df.iloc[:,-1]
d=initialize_dict()
df_meta=df_meta.append(update_dict(X,y),ignore_index=True)
df_meta.iloc[-1,0]='poker hand'

### contraceptive

In [11]:
df=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/cmc/cmc.data',header=None)
df.head(3)

,0,1,2,3,4,5,6,7,8,9
0,24,2,3,3,1,1,2,3,0,1
1,45,1,3,10,1,1,3,4,0,1
2,43,2,3,7,1,1,3,4,0,1


In [12]:
df[1]=df[0].astype(str)
df[2]=df[2].astype(str)
df[6]=df[6].astype(str)
df[7]=df[7].astype(str)
df[9]=df[9].astype(str)

X=df.iloc[:,:-1]
y=df.iloc[:,-1]
d=initialize_dict()
df_meta=df_meta.append(update_dict(X,y),ignore_index=True)
df_meta.iloc[-1,0]='contraceptive'

In [13]:
df_meta

,dataset,n_samples,n_features,n_categorical,n_numerical,highest_cardinality,nulls,n_classes,classes,class_ratio,imbalanced,domain,xgboost_acc,catboost_acc,lgbm_acc,RF_acc
0,amazon,32769,9,9,0,7518,0.0,2,"[1, 0]","[0.9421, 0.0579]",1,None,0,0,0,0
1,contraceptive,1473,9,4,5,34,0.0,3,"[1, 3, 2]","[0.427, 0.3469, 0.2261]",1,None,0,0,0,0


### census_income

In [ ]:
df=pd.read_csv('F:/data/income/census-income.data',header=None)
df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,73,Not in universe,0,0,High school graduate,0,Not in universe,Widowed,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,0,95,- 50000.
1,58,Self-employed-not incorporated,4,34,Some college but no degree,0,Not in universe,Divorced,Construction,Precision production craft & repair,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,52,94,- 50000.
2,18,Not in universe,0,0,10th grade,0,High school,Never married,Not in universe or children,Not in universe,...,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0,Not in universe,2,0,95,- 50000.


In [ ]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]
d=initialize_dict()
df_meta=df_meta.append(update_dict(X,y),ignore_index=True)
df_meta.iloc[-1,0]='census_income'

### amazon

In [7]:
df=pd.read_csv('train.csv',dtype='str')
df.head(3)

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880


In [9]:
X=df.iloc[:,1:]
y=df.iloc[:,0]
d=initialize_dict()
df_meta=df_meta.append(update_dict(X,y),ignore_index=True)
df_meta.iloc[-1,0]='amazon'

### kdd appet

In [9]:
df=pd.read_csv('../small_train.data',delimiter='\t')
df.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
0,NaN,NaN,NaN,NaN,NaN,1526.0,7.0,NaN,NaN,NaN,...,oslk,fXVEsaq,jySVZNlOJy,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,525.0,0.0,NaN,NaN,NaN,...,oslk,2Kb5FSF,LM8l689qOp,NaN,NaN,fKCe,RAYp,F2FyR07IdsN7I,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,5236.0,7.0,NaN,NaN,NaN,...,Al6ZaUT,NKv4yOc,jySVZNlOJy,NaN,kG3k,Qu4f,02N6s8f,ib5G6X1eUxUn6,am7c,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,oslk,CE7uk3u,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,1029.0,7.0,NaN,NaN,NaN,...,oslk,1J2cvxe,LM8l689qOp,NaN,kG3k,FSa2,RAYp,F2FyR07IdsN7I,mj86,NaN


In [13]:
y=pd.read_csv('../small_train.labels',header=None)

In [17]:
X=df.iloc[:,:]
y=y.iloc[:,0]
d=initialize_dict()
df_meta=df_meta.append(update_dict(X,y),ignore_index=True)
df_meta.iloc[-1,0]='kdd appet'

In [18]:
df_meta

,dataset,n_samples,n_features,n_categorical,n_numerical,highest_cardinality,nulls,n_classes,classes,class_ratio,imbalanced,domain,xgboost_acc,catboost_acc,lgbm_acc,RF_acc
0,kdd appet,50000,230,38,192,15415,0.698,2,"[-1, 1]","[0.9822, 0.0178]",1,None,0,0,0,0


### healthcare provider

In [77]:
df=pd.read_csv('C:\\Users\\nishant\\Documents\\Data Viz\\3\\provider.csv')
df=df[[  'nppes_credentials', 'nppes_provider_gender',
       'nppes_provider_city', 'nppes_provider_zip5',
       'nppes_provider_state', 'nppes_provider_country',
       'specialty_description', 'medicare_prvdr_enroll_status','bene_count']]

df['nppes_provider_zip5']=df['nppes_provider_zip5'].astype(str)
df.dropna(inplace=True)

df['n_patients']=pd.qcut(df.bene_count,4)

In [78]:
X=df.iloc[:,:-2]
y=df.iloc[:,-1]
d=initialize_dict()
df_meta=df_meta.append(update_dict(X,y),ignore_index=True)
df_meta.iloc[-1,0]='healthcare provider'

In [79]:
df_meta

,dataset,n_samples,n_features,n_categorical,n_numerical,highest_cardinality,nulls,n_classes,classes,class_ratio,imbalanced,domain,xgboost_acc,catboost_acc,lgbm_acc,RF_acc
0,healthcare provider,969617,8,8,0,19574,0.0,4,"[(10.999, 35.0], (89.0, 216.0], (216.0, 39965....","[0.252, 0.2502, 0.2497, 0.2481]",1,None,0,0,0,0
1,healthcare provider,969617,8,8,0,19574,0.0,4,"[(10.999, 35.0], (89.0, 216.0], (216.0, 39965....","[0.252, 0.2502, 0.2497, 0.2481]",1,None,0,0,0,0
2,healthcare provider,969617,8,8,0,19574,0.0,4,"[(10.999, 89.0], (89.0, 216.0], (216.0, 406.0]...","[0.5001, 0.2502, 0.1497, 0.1]",1,None,0,0,0,0
3,healthcare provider,969617,8,8,0,19574,0.0,4,"[(10.999, 35.0], (89.0, 216.0], (216.0, 39965....","[0.252, 0.2502, 0.2497, 0.2481]",1,None,0,0,0,0
4,healthcare provider,969617,8,8,0,19574,0.0,4,"[(10.999, 35.0], (89.0, 216.0], (216.0, 39965....","[0.252, 0.2502, 0.2497, 0.2481]",1,None,0,0,0,0


### land prices

In [106]:
df=pd.read_csv('../agri_land.csv')

In [107]:
df.dropna(inplace=True)
df['Acre Value']=pd.Series(pd.cut(df.iloc[:,-1],10)).astype(str)

In [108]:
df.head()

,State,LandCategory,Region,Region or State,Year,Acre Value
0,Northeast,Farm Real Estate,Northeast,Region,1997,"(1806.9, 3472.8]"
1,Connecticut,Farm Real Estate,Northeast,State,1997,"(5138.7, 6804.6]"
2,Delaware,Farm Real Estate,Northeast,State,1997,"(1806.9, 3472.8]"
3,Maine,Farm Real Estate,Northeast,State,1997,"(124.341, 1806.9]"
4,Maryland,Farm Real Estate,Northeast,State,1997,"(1806.9, 3472.8]"


In [109]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [110]:
d=initialize_dict()
df_meta=df_meta.append(update_dict(X,y),ignore_index=True)
df_meta.iloc[-1,0]='agri prices'

In [111]:
df_meta

,dataset,n_samples,n_features,n_categorical,n_numerical,highest_cardinality,nulls,n_classes,classes,class_ratio,imbalanced,domain,xgboost_acc,catboost_acc,lgbm_acc,RF_acc
0,agri prices,3458,5,4,1,60,0.0,10,"[(124.341, 1806.9], (1806.9, 3472.8], (3472.8,...","[0.4378, 0.3242, 0.1217, 0.0468, 0.0318, 0.011...",1,None,0,0,0,0


In [113]:
df_meta.to_csv('a.csv')

### test viability

In [76]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [77]:
import lightgbm as lgbm
from sklearn.metrics import accuracy_score, f1_score,jaccard_score,multilabel_confusion_matrix,log_loss
from catboost import CatBoostClassifier
from xgboost import XGBClassifier


model=CatBoostClassifier(task_type="GPU", devices='0:1',iterations=100)
model.fit(X_train,y_train,cat_features=df.select_dtypes('object'))

pred=model.predict(X_test)
pred_proba=model.predict_proba(X_test)

print('accuracy:',accuracy_score(y_test,pred))
print('f1 macro:',f1_score(y_test,pred, average='macro'))
# print('f1_micro:',f1_score(y_test,pred, average='micro'))
#     print(pd.Series(pred).unique())
print('log_loss:',log_loss(y_test,pred_proba))


Learning rate set to 0.5
0:	learn: 1.2828261	total: 17.8ms	remaining: 1.76s
1:	learn: 1.0551750	total: 33ms	remaining: 1.62s
2:	learn: 0.9478717	total: 47.4ms	remaining: 1.53s
3:	learn: 0.8877344	total: 63.6ms	remaining: 1.53s
4:	learn: 0.8321970	total: 78.5ms	remaining: 1.49s
5:	learn: 0.7778258	total: 93.7ms	remaining: 1.47s
6:	learn: 0.7474037	total: 109ms	remaining: 1.44s
7:	learn: 0.7215865	total: 125ms	remaining: 1.44s
8:	learn: 0.6916493	total: 138ms	remaining: 1.4s
9:	learn: 0.6736466	total: 152ms	remaining: 1.36s
10:	learn: 0.6483796	total: 165ms	remaining: 1.33s
11:	learn: 0.6293597	total: 178ms	remaining: 1.3s
12:	learn: 0.6137893	total: 190ms	remaining: 1.27s
13:	learn: 0.5990171	total: 202ms	remaining: 1.24s
14:	learn: 0.5750886	total: 216ms	remaining: 1.23s
15:	learn: 0.5577179	total: 232ms	remaining: 1.22s
16:	learn: 0.5442012	total: 249ms	remaining: 1.21s
17:	learn: 0.5315259	total: 263ms	remaining: 1.2s
18:	learn: 0.5229190	total: 276ms	remaining: 1.18s
19:	learn: 0.51

In [78]:
yy=[[0.460983,0.287572,0.144509,0.043353,0.020231,0.015896,0.007225,0.007225,0.007225,0.00578]for i in range(len(y_test))]

In [79]:
log_loss(y_test,yy)

3.2732964042009036

In [44]:
y_test.value_counts()/y_test.value_counts().sum()

(124.341, 1806.9]     0.460983
(1806.9, 3472.8]      0.287572
(3472.8, 5138.7]      0.144509
(5138.7, 6804.6]      0.043353
(6804.6, 8470.5]      0.020231
(8470.5, 10136.4]     0.015896
(15134.1, 16800.0]    0.007225
(13468.2, 15134.1]    0.007225
(11802.3, 13468.2]    0.007225
(10136.4, 11802.3]    0.005780
Name: Acre Value, dtype: float64